In [4]:
from pynwb import NWBFile
from datetime import datetime

In [5]:
import h5py as h5
import numpy as np

In [6]:
nwb1_file = '/Users/xiaoxuanj/work/work_allen/Ephys/mouse366119/mouse366119_probe_surf.nwb'
nwb1 = h5.File(nwb1_file)

In [4]:
list(nwb1['processing'])

['probeA', 'probeB', 'probeC', 'probeD', 'probeE', 'probeF']

In [5]:
list(nwb1['processing']['probeA']['Clustering'])

['cluster_nums', 'num', 'peak_over_rms', 'times']

In [6]:
nwb1['general']['experimenter'].value

b'experimenter'

In [7]:
nwb1['general']['subject']['subject_id'].value

b'subject_id'

In [8]:
nwb1['general']['experiment_description'].value.astype(str)

'with visual stimuli; awake; sorted; unhealthy brain; tail touching grounding; probe C has sync recorded'

In [9]:
str(nwb1['identifier'].value)

"b'366119; NWB-1.0.5; Wed Mar 21 15:17:53 2018'"

# create nwb instance and add info to general

In [10]:
nwb2 = NWBFile('NWB 1.0 > NWB 2.0 converter',        # source
                  str(nwb1['session_description'].value), # session description
                  str(nwb1['identifier'].value),          # identifier
                  str(nwb1['session_start_time'].value.astype(str)),  # date
                  experimenter = nwb1['general']['experimenter'].value.astype(str),
                  lab = nwb1['general']['lab'].value.astype(str),
                  institution = nwb1['general']['lab'].value.astype(str),
                  experiment_description = nwb1['general']['experiment_description'].value.astype(str),
                  session_id='SESSION_ID')

In [11]:
device = nwb2.create_device(name='Neuropixels Prototype Rig', source="a source")

In [12]:
probes = [p for p in nwb1['processing'].keys() if p[:5] == 'probe']

In [13]:
probes

['probeA', 'probeB', 'probeC', 'probeD', 'probeE', 'probeF']

In [14]:
locations = {'probeA' : 'AM',
             'probeB' : 'PM',
             'probeC' : 'V1',
             'probeD' : 'LM',
             'probeE' : 'AL',
             'probeF' : 'RL'}

xlocs = [16, 48, 0, 32]

In [15]:
for probe in probes:

    electrode_name = probe
    source = "Neuropixels Probe"
    description = "Phase 3a"
    location = locations[probe]

    electrode_group = nwb2.create_electrode_group(electrode_name,
                                                 source=source,
                                                 description=description,
                                                 location=location,
                                                 device=device)
    
    for idx in range(0,384):
        nwb2.add_electrode(idx,
                          x=float(xlocs[idx % 4]), y=float(idx/2*20), z=np.nan,
                          imp=np.nan,
                          location='unknown', filtering='300 Hz highpass',
                          description='channel %s' % idx, group=electrode_group)

In [16]:
list(nwb1['processing']['probeA']['UnitTimes']['86'])

['channel',
 'snr',
 'source',
 'template',
 'times',
 'type',
 'unit_description',
 'waveform',
 'xpos_probe',
 'ypos_probe']

# add unit times to processing

In [18]:
from pynwb.misc import UnitTimes

module = nwb2.create_processing_module('ecephys', # name
                                       'ecephys', #source
                                        'UnitTimes module' # description
                                      )
for probe in probes:
    units = []
    spike_times=[]
    unittimes = UnitTimes(name=probe, source=source) # name
    for unit in list(nwb1['processing'][probe]['UnitTimes']):
        if unit!='noise' and unit!= 'unit_list':
            unittimes.add_spike_times(int(unit.encode('utf8')), list(nwb1['processing'][probe]['UnitTimes'][unit]['times']))
          #  units.append(unit.encode('utf8'))
          #  spike_times.append(list(nwb1['processing']['probeA']['UnitTimes'][unit]['times']))
            
    
    #unittimes = UnitTimes(probe, # source
    #                      unit_ids=units, #unit_ids
    #                      spike_times=spike_times, # list of times for correponding units
    #                      spike_times_index = [], #the indices in spike_times that correspond to each unit in unit_ids
    #                      name='UnitTimes') # name

    module.add_data_interface(unittimes)                               


In [19]:
probe

'probeA'

In [21]:
module.containers

(<pynwb.misc.UnitTimes at 0x108fc8b38>,)

# add stimulus to stimulus group

In [20]:
# opto tagging is also here
from pynwb.misc import AbstractFeatureSeries

for stimulus in nwb1['stimulus']['presentation'].keys():
    afs = AbstractFeatureSeries(stimulus, # name
                          stimulus, # source
                          nwb1['stimulus']['presentation'][stimulus]['feature_units'].value, # units
                          nwb1['stimulus']['presentation'][stimulus]['features'].value, # features
                          data = nwb1['stimulus']['presentation'][stimulus]['data'].value, # data
                          timestamps = nwb1['stimulus']['presentation'][stimulus]['timestamps'].value)
    
    nwb2.add_stimulus(afs)

# add movie to stimulus template

In [21]:
from pynwb.misc import AbstractFeatureSeries

# add movie
# ophys movie
moviepath='/Users/xiaoxuanj/work/work_allen/visual_stimuli/movie'

stimulus_movie = np.load('/Users/xiaoxuanj/work/work_allen/visual_stimuli/movie/Movie_TOE1.npy')

afs = AbstractFeatureSeries(data = stimulus_movie, # name
                            source = 'movie',
                            features=['time-space-space'],
                            feature_units=['luminance'],
                            rate = 30.,
                            starting_time = 0.,
                           name = 'natural_movie_ephys')

nwb2.add_stimulus_template(afs)

# add running speed

In [ ]:
from pynwb.behavior import BehavioralEpochs

running = BehavioralEpochs(source='run', interval_series={}, name='BehavioralEpochs')

running.create_interval_series(name='behavior_run', source='behavior_run', data=[], timestamps=[], comments='no comments', 
                       description='no description', control=None, control_description=None)

nwb2.add_interval_series(running)

# add image

In [ ]:
def add_image(self, stream, name, fmt, desc, dtype=None):
    """Add aquisition images: probe map, insersion map, and retinotopic map.
       Adds documentation image (or movie) to file. This is stored in /acquisition/images/.

        Args:
            *stream* (binary) Data stream of image (eg, binary contents of .png image)

            *name* (text) Name that image will be stored as

            *fmt* (text) Format of the image (eg, "png", "avi")

            *desc* (text) Descriptive text describing the image

            *dtype* (text) Optional field specifying the h5py datatype to use to store *stream*

        Returns:
            *nothing*
    """

    abstract = self.create_reference_image(stream, name, fmt, desc, dtype=None)

# add LFP

In [ ]:
def add_lfp(self, data, times=np.array([0]),start_time=0.0,source='iMec',**kwargs):
    """Add filtered LFP as TimeSeries extension. Array structure: [num time samples][num channels]
    Todo: add sampling_rate as abstract.append_??"""

    if 'name' in kwargs.keys():
        name = kwargs['name']
    else:
        name = 'LFP'

    abstract = self.create_timeseries("ElectricalSeries",name, "acquisition",)

    abstract.set_data(data, resolution=1./2500)
    abstract.set_source(source)
    abstract.set_time(times)
    abstract.set_value("num_samples", np.shape(data)[0])
    abstract.set_value("electrode_idx", [0,1])
    abstract.set_value("sampling_rate", 2500)
    abstract.finalize()



In [ ]:
from pynwb.ecephys import LFP

lfp = LFP(source, electrical_series={}, name='LFP') # name

lfp.create_electrical_series(name=probename, source=probename, data, electrodes, resolution=0.0, conversion=1.0, timestamps=None, 
                         starting_time=None, rate=None, comments='no comments', description='no description', control=None, control_description=None)

In [ ]:
pynwb.ecephys.LFP(source, electrical_series={}, name='LFP')

# save file to NWB

In [23]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_pipeline_test/nwb/probeA_new.nwb', 'w') as io:
    io.write(nwb2)

# read file with NWB IO

In [25]:
io = NWBHDF5IO('/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_pipeline_test/nwb/stimulus_new.nwb', 'r')
nwbfile = io.read()

In [26]:
stimulus = list(nwbfile.stimulus)

In [27]:
stimulus

['flash_250ms_1', 'gabor_20_deg_0', 'perceptual_grouping02_2', 'spontaneous']

In [28]:
movie = nwbfile.get_stimulus_template('natural_movie_ephys').data[:]